# Lesson 3: Reflection and Blogpost Writing

## Setup

In [1]:
llm_config = {"model": "gpt-3.5-turbo"}

## The task!

In [2]:
task = '''
        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       '''


## Create a writer agent

In [3]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [4]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [5]:
print(reply)

Title: "Unleashing the Power of AI with DeepLearning.AI"

Embark on a journey to unlock the secrets of artificial intelligence with DeepLearning.AI. Founded by renowned data scientist Andrew Ng, this platform offers cutting-edge courses on deep learning to empower individuals and organizations worldwide. Whether you're a beginner or an expert, these engaging and comprehensive courses will equip you with the skills needed to thrive in the AI-driven future. From computer vision to natural language processing, DeepLearning.AI covers it all. Join this transformative learning experience today and dive into the exciting world of AI with confidence. Let's shape a smarter tomorrow, one algorithm at a time.


## Adding reflection 

Create a critic agent to reflect on the work of the writer agent.

In [6]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [7]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: "Unleashing the Power of AI with DeepLearning.AI"

Embark on a journey to unlock the secrets of artificial intelligence with DeepLearning.AI. Founded by renowned data scientist Andrew Ng, this platform offers cutting-edge courses on deep learning to empower individuals and organizations worldwide. Whether you're a beginner or an expert, these engaging and comprehensive courses will equip you with the skills needed to thrive in the AI-driven future. From computer vision to natural language processing, DeepLearning.AI covers it all. Join this transformative learning experience today and dive into the exciting world of AI with confidence. Let's shape a smarter tomorrow, one algorithm at a time.

-------------------------------

## Nested chat

In [8]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


In [9]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [10]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [11]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve the task

In [12]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [13]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different task.

In [14]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: "Unleashing the Power of AI with DeepLearning.AI"

Embark on a journey to unlock the secrets of artificial intelligence with DeepLearning.AI. Founded by renowned data scientist Andrew Ng, this platform offers cutting-edge courses on deep learning to empower individuals and organizations worldwide. Whether you're a beginner or an expert, these engaging and comprehensive courses will equip you with the skills needed to thrive in the AI-driven future. From computer vision to natural language processing, DeepLearning.AI covers it all. Join this transformative learning experience today and dive into the exciting world of AI with confidence. Let's shape a smarter tomorrow, one algorithm at a time.

-------------------------------


--------------------------------------------------------------------------------
Meta Reviewer (to Critic):

Aggregated feedback:

SEO Reviewer:
1. Optimize the title for better search engine visibility by including relevant keywords.
2. Include internal links to related content on the DeepLearning.AI website for improved site structure and user engagement.
3. Enhance the meta description with a clear call-to-action and focus on key AI topics to attract organic traffic.

Ethics Reviewer:
1. Ensure clear disclosure of any biases or conflicts of interest related to DeepLearning.AI and Andrew Ng.
2. Verify the accuracy of claims about the courses' effectiveness.
3. Check that marketing language doesn't overstate benefits or mislead learners about the skills they will gain.

Final suggestions:
1. Incorporate relevant keywords in the title to boost search engine visibility. Consider the suggestion of "Master Artificial Intelligence with DeepLearning.AI: Courses by Andrew Ng."
2. Include in

## Get the summary

In [15]:
print(res.summary)

Title: "Master Artificial Intelligence with DeepLearning.AI: Courses by Andrew Ng"

Unlock the world of AI through DeepLearning.AI, led by the esteemed Andrew Ng. Dive into comprehensive courses covering key AI topics, from computer vision to natural language processing. Our platform caters to all skill levels, offering transformative learning experiences for a smarter tomorrow. Enhance your expertise and stay ahead in this AI-driven era. For more insights, explore related content on the DeepLearning.AI website and join us in shaping the future of technology. Full disclosure, verified effectiveness, and precise marketing – trust us to guide you accurately through the realm of AI.
